In [ ]:
# ============================================
# TRAVELMATE AI CHATBOT - FULL LLAMA LLM VERSION
# Using Real Llama Model for Natural Language Understanding
# ============================================

print("🚀 Starting TravelMate AI with Llama LLM...")
print("=" * 60)

# ============================================
# PILIH MODE: API atau LOCAL
# ============================================
USE_API = True  # Set False untuk pakai model local (download 4GB+)

# STEP 1: Install Dependencies
print("\n📦 Installing dependencies...")
if USE_API:
    print("   Mode: API (Fast & Efficient)")
    !pip install -q gradio requests
else:
    print("   Mode: LOCAL (Downloading model...)")
    !pip install -q gradio transformers torch accelerate bitsandbytes sentencepiece

import gradio as gr
from datetime import datetime
import json
import random

if USE_API:
    import requests
else:
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
    print("   📥 This will download ~4GB model...")

print("✅ Dependencies installed!\n")

# ============================================
# CONFIGURATION
# ============================================

# API Configuration (if using API mode)
API_CONFIG = {
    "groq": {
        "url": "https://api.groq.com/openai/v1/chat/completions",
        "model": "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",  # Fast Llama 3.1
        "api_key": "YOUR_GROQ_API_KEY_HERE"  # Get free at: https://console.groq.com
    },
    "together": {
        "url": "https://api.together.xyz/v1/chat/completions",
        "model": "meta-llama/Llama-3-70b-chat-hf",
        "api_key": "YOUR_TOGETHER_API_KEY_HERE"  # Get free at: https://api.together.xyz
    }
}

# Local Model Configuration
LOCAL_MODEL = "meta-llama/Llama-2-7b-chat-hf"  # Alternatif: "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# System Prompt for Llama
SYSTEM_PROMPT = """You are TravelMate AI, a friendly and knowledgeable travel assistant. Your characteristics:

PERSONALITY:
- Warm, enthusiastic, and helpful
- Use casual Indonesian language with occasional English terms
- Add relevant emojis to make conversations lively
- Be encouraging and motivational about travel

EXPERTISE:
- Travel destinations worldwide (especially Asia & Indonesia)
- Budget planning and cost estimation
- Itinerary creation (day-by-day plans)
- Local cuisine recommendations
- Transportation and visa information
- Cultural insights and travel tips
- Hidden gems and Instagram-worthy spots

RESPONSE STYLE:
- Start with enthusiasm and acknowledgment
- Provide structured, easy-to-read information
- Use bullet points, emojis, and formatting
- Always ask follow-up questions to better assist
- Give specific, actionable advice
- Include budget estimates when relevant

KNOWLEDGE BASE:
- Bali: Ubud, Seminyak, Uluwatu, Nusa Penida (Budget: 300k-1jt/day)
- Tokyo: Shibuya, Harajuku, Asakusa (Budget: ¥10k-15k/day)
- Bangkok: Grand Palace, Chatuchak, street food (Budget: ฿1k-2k/day)
- Yogyakarta: Borobudur, Prambanan, Malioboro (Budget: 150k-500k/day)
- Singapore: Marina Bay, Gardens by the Bay (Budget: SGD 80-150/day)

Always be helpful, never refuse travel questions, and make trip planning exciting!"""

# ============================================
# LLAMA MODEL INITIALIZATION
# ============================================

class LlamaModel:
    def __init__(self, use_api=True, api_provider="groq"):
        self.use_api = use_api
        self.api_provider = api_provider
        self.model = None
        self.tokenizer = None
        self.chat_history = []

        if use_api:
            self.setup_api()
        else:
            self.setup_local()

    def setup_api(self):
        """Setup API-based Llama"""
        print(f"\n🔗 Setting up Llama via {self.api_provider.upper()} API...")
        config = API_CONFIG[self.api_provider]

        if "YOUR_" in config["api_key"]:
            print("⚠️  WARNING: No API key detected!")
            print(f"   Get free API key at: https://console.groq.com (Groq)")
            print("   Or: https://api.together.xyz (Together AI)")
            print("\n   💡 Update API_CONFIG['groq']['api_key'] in code")
            print("   For now, using fallback responses...\n")
            self.use_fallback = True
        else:
            self.use_fallback = False
            print("✅ API configured successfully!")

    def setup_local(self):
        """Setup local Llama model"""
        print(f"\n🤖 Loading local Llama model: {LOCAL_MODEL}")
        print("   This may take 5-10 minutes on first run...")

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL)

            # Load with 4-bit quantization to save memory
            self.model = AutoModelForCausalLM.from_pretrained(
                LOCAL_MODEL,
                device_map="auto",
                torch_dtype=torch.float16,
                load_in_4bit=True,  # 4-bit quantization
                low_cpu_mem_usage=True
            )

            print("✅ Model loaded successfully!")
            self.use_fallback = False

        except Exception as e:
            print(f"❌ Error loading model: {e}")
            print("   Falling back to rule-based responses...")
            self.use_fallback = True

    def generate_api_response(self, user_message):
        """Generate response using API"""
        if self.use_fallback:
            return self.fallback_response(user_message)

        try:
            config = API_CONFIG[self.api_provider]

            # Prepare messages
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT}
            ]

            # Add chat history (last 5 messages)
            for msg in self.chat_history[-5:]:
                messages.append({"role": msg["role"], "content": msg["content"]})

            messages.append({"role": "user", "content": user_message})

            # API request
            headers = {
                "Authorization": f"Bearer {config['api_key']}",
                "Content-Type": "application/json"
            }

            data = {
                "model": config["model"],
                "messages": messages,
                "temperature": 0.7,
                "max_tokens": 500,
                "top_p": 0.9
            }

            response = requests.post(
                config["url"],
                headers=headers,
                json=data,
                timeout=30
            )

            if response.status_code == 200:
                result = response.json()
                return result["choices"][0]["message"]["content"]
            else:
                print(f"API Error: {response.status_code}")
                return self.fallback_response(user_message)

        except Exception as e:
            print(f"Error: {e}")
            return self.fallback_response(user_message)

    def generate_local_response(self, user_message):
        """Generate response using local model"""
        if self.use_fallback:
            return self.fallback_response(user_message)

        try:
            # Format prompt for Llama-2
            prompt = f"<s>[INST] <<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n\n"

            # Add history
            for msg in self.chat_history[-3:]:
                if msg["role"] == "user":
                    prompt += f"{msg['content']} [/INST] "
                else:
                    prompt += f"{msg['content']} </s><s>[INST] "

            prompt += f"{user_message} [/INST]"

            # Generate
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=400,
                    temperature=0.7,
                    top_p=0.9,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract only the response part
            response = response.split("[/INST]")[-1].strip()

            return response

        except Exception as e:
            print(f"Generation error: {e}")
            return self.fallback_response(user_message)

    def fallback_response(self, user_message):
        """Intelligent fallback when API/model unavailable"""
        msg_lower = user_message.lower()

        # Knowledge base responses
        destinations = {
            "bali": "🏖️ **Bali** adalah destinasi perfect!\n\n📍 **Must-visit:** Ubud (rice terraces), Uluwatu (temple), Seminyak (beach clubs), Nusa Penida (Kelingking beach)\n\n🍜 **Kuliner:** Babi Guling, Nasi Campur, Sate Lilit\n\n💰 **Budget:** Rp 300k-1jt/hari\n\n📅 **Best time:** April-Oktober\n\nMau itinerary detail berapa hari? 😊",

            "tokyo": "🗼 **Tokyo** - perpaduan tradisi & modern!\n\n📍 **Areas:** Shibuya (crossing), Harajuku (fashion), Asakusa (Sensoji), Akihabara (anime)\n\n🍣 **Food:** Sushi, ramen, takoyaki, matcha desserts\n\n💰 **Budget:** ¥10,000-15,000/hari\n\n🚇 **Tip:** Beli Suica card untuk transport!\n\nMau info lebih detail tentang apa nih?",

            "bangkok": "🛕 **Bangkok** - street food heaven!\n\n📍 **Highlights:** Grand Palace, Wat Arun, Chatuchak Market, Khao San Road\n\n🍜 **Must-try:** Pad Thai, Tom Yum, Mango Sticky Rice\n\n💰 **Budget:** ฿1,000-2,000/hari (super murah!)\n\n💡 **Tip:** Naik BTS Skytrain untuk avoid macet!\n\nAda yang mau ditanya lebih lanjut?",

            "yogyakarta": "🏛️ **Yogyakarta** - heart of Javanese culture!\n\n📍 **Must-see:** Borobudur (sunrise!), Prambanan, Malioboro, Keraton\n\n🍜 **Kuliner:** Gudeg Yu Djum, Bakpia, Sate Klathak\n\n💰 **Budget:** Rp 150k-500k/hari\n\n📅 **Perfect for:** 3-4 hari trip\n\nMau itinerary lengkap?"
        }

        for dest, info in destinations.items():
            if dest in msg_lower:
                return info

        # Budget recommendations
        if any(word in msg_lower for word in ['budget', 'rekomendasi', 'suggest', 'recommend']):
            return """💰 **Travel Recommendations by Budget:**

**🎒 Budget (<3 juta):**
• Yogyakarta (Rp 1.5-2jt)
• Bandung (Rp 1.5-2.5jt)
• Malang (Rp 2-2.5jt)

**✈️ Mid-Range (3-7 juta):**
• Bali (Rp 3.5-5jt)
• Bangkok (Rp 3-4.5jt)
• Singapore (Rp 5-6.5jt)

**🌏 Premium (>7 juta):**
• Tokyo (Rp 12-15jt)
• Seoul (Rp 10-13jt)
• Malaysia (Rp 7-9jt)

Budget kamu sekitar berapa? Biar aku kasih rekomendasi yang pas! 🎯"""

        # Itinerary request
        if 'itinerary' in msg_lower or 'jadwal' in msg_lower:
            return """📅 **Itinerary Planning**

Oke siap! Untuk buatin itinerary yang pas, aku perlu info:

1️⃣ **Destinasi:** Mau ke mana?
2️⃣ **Durasi:** Berapa hari?
3️⃣ **Budget:** Range budget kamu?
4️⃣ **Interests:** Prefer culture/beach/adventure/food?

Share detail-nya, aku buatin jadwal lengkap! 🗺️"""

        # Default engaging response
        responses = [
            f"Interesting question tentang travel! 🌏 Untuk '{user_message}', aku bisa bantu lebih detail kalau kamu cerita:\n\n• Destinasi yang kamu pertimbangkan?\n• Budget range berapa?\n• Durasi trip berapa hari?\n\nShare dong biar aku bisa kasih rekomendasi yang perfect! ✨",

            f"Wah, menarik nih! Tentang '{user_message}' - setiap traveler punya kebutuhan berbeda. \n\nCerita lebih banyak dong tentang preferensi kamu:\n📍 Prefer city atau nature?\n🎒 Travel style: santai atau adventure?\n💰 Budget yang kamu alokasikan?\n\nBiar rekomendasiku lebih personal! 😊"
        ]

        return random.choice(responses)

    def chat(self, user_message):
        """Main chat function"""
        # Add to history
        self.chat_history.append({
            "role": "user",
            "content": user_message
        })

        # Generate response
        if self.use_api:
            response = self.generate_api_response(user_message)
        else:
            response = self.generate_local_response(user_message)

        # Add response to history
        self.chat_history.append({
            "role": "assistant",
            "content": response
        })

        return response

# ============================================
# INITIALIZE LLAMA
# ============================================
print("\n" + "=" * 60)
print("🤖 Initializing Llama Model...")
print("=" * 60)

llama_bot = LlamaModel(use_api=USE_API, api_provider="groq")

print("\n✅ TravelMate AI is ready!")
print("=" * 60 + "\n")

# ============================================
# GRADIO INTERFACE
# ============================================

def chat_interface(message, history):
    """Gradio chat function"""
    if not message.strip():
        return history, ""

    # Generate response using Llama
    response = llama_bot.chat(message)

    # Update history
    history.append([message, response])

    return history, ""

def create_ui():
    with gr.Blocks(
        theme=gr.themes.Soft(primary_hue="purple", secondary_hue="blue"),
        css="""
        .gradio-container {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        }
        """
    ) as demo:

        gr.Markdown(f"""
        # ✈️ TravelMate AI - Powered by Llama LLM
        ### 🤖 Using Real {"API" if USE_API else "Local"} Llama Model for Intelligent Conversations

        Asisten travel pintar yang memahami natural language! 🗺️
        """)

        if USE_API and llama_bot.use_fallback:
            gr.Markdown("""
            ⚠️ **API Key Required:** Update `API_CONFIG` with your free API key from:
            - Groq: https://console.groq.com (Recommended - Fast!)
            - Together AI: https://api.together.xyz

            Currently using fallback mode with limited capabilities.
            """)

        with gr.Row():
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(
                    value=[[None, "✈️ Halo traveler! Aku TravelMate AI powered by Llama LLM! 🤖\n\nAku bisa bantu kamu dengan:\n• Planning destinasi & itinerary\n• Budget calculation\n• Kuliner recommendations\n• Travel tips & hacks\n\nMau jalan-jalan ke mana nih? Cerita aja, aku bakal ngerti! 🗺️"]],
                    height=500,
                    bubble_full_width=False
                )

                with gr.Row():
                    msg = gr.Textbox(
                        placeholder="Tanya apa aja tentang travel... Llama akan memahami natural language kamu!",
                        show_label=False,
                        scale=4
                    )
                    send_btn = gr.Button("Send 🚀", scale=1, variant="primary")

                gr.Examples(
                    examples=[
                        "Aku mau liburan 5 hari budget 5 juta, recommend destinasi dong",
                        "Gimana caranya traveling ke Jepang dengan budget minimal?",
                        "Bikin itinerary 3 hari di Bali yang hits untuk Instagram",
                        "Apa aja kuliner wajib coba kalau ke Bangkok?",
                        "Tips backpacking Asia Tenggara untuk pemula"
                    ],
                    inputs=msg,
                    label="💬 Contoh Natural Language Queries"
                )

            with gr.Column(scale=1):
                gr.Markdown(f"""
                ### 🎯 Model Info

                **Engine:** {"Llama 3.1 70B (API)" if USE_API else "Llama 2 7B (Local)"}
                **Provider:** {("Groq API" if llama_bot.api_provider == "groq" else "Together AI") if USE_API else "Transformers"}
                **Status:** {"⚠️ Fallback Mode" if llama_bot.use_fallback else "✅ Active"}

                ### 📚 Capabilities

                - 🧠 Natural language understanding
                - 💬 Context-aware conversations
                - 🎯 Personalized recommendations
                - 📊 Structured responses
                - 🌏 Multi-destination knowledge

                ### 💡 Tips

                - Tanya dengan natural language
                - Sebutkan budget & durasi
                - Cerita preferensi kamu
                - Follow-up untuk detail

                **Destination Coverage:**
                🏝️ Bali | 🗼 Tokyo | 🛕 Bangkok
                🏛️ Yogyakarta | 🦁 Singapore
                🇰🇷 Seoul | 🇻🇳 Vietnam | & more!
                """)

        # Event handlers
        msg.submit(chat_interface, [msg, chatbot], [chatbot, msg])
        send_btn.click(chat_interface, [msg, chatbot], [chatbot, msg])

        gr.Markdown("""
        ---
        **🤖 Powered by Llama LLM** | Built for intelligent travel assistance

        💡 Llama model memberikan pemahaman natural language yang lebih baik dibanding rule-based chatbot!
        """)

    return demo

# ============================================
# LAUNCH
# ============================================

print("🚀 Launching TravelMate AI Chatbot...")
print("=" * 60)

demo = create_ui()
demo.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)

print("\n✅ Chatbot is running with Llama LLM!")
print("🔗 Click the public link above to start chatting!")
print("\n" + "=" * 60)

🚀 Starting TravelMate AI with Llama LLM...

📦 Installing dependencies...
   Mode: API (Fast & Efficient)
✅ Dependencies installed!


🤖 Initializing Llama Model...

🔗 Setting up Llama via GROQ API...
⚠️  WARNING: No API key detected!
   Get free API key at: https://console.groq.com (Groq)
   Or: https://api.together.xyz (Together AI)

   💡 Update API_CONFIG['groq']['api_key'] in code
   For now, using fallback responses...


✅ TravelMate AI is ready!

🚀 Launching TravelMate AI Chatbot...


/tmp/ipython-input-768437135.py:385: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-768437135.py:385: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://727562ee0c8b0238c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
